In [66]:
import numpy as np
import pywt
import matplotlib.pyplot as plt
import nibabel as nib
import random
from scipy import stats
from scipy import fft
import random
import pywt.data
from PIL import Image
import pandas as pd
import seaborn as sns
import os
import pickle

In [67]:
# assign directory
import os
BRANDON = '/Users/brandonmarks/Desktop/Research Materials/hierarchical-bayesian-model-validation/'
YASH = '/Users/yashd/Desktop/hierarchical-bayesian-model-validation/'

ROOT_DIR = BRANDON
data_dir = ROOT_DIR + 'raw-data/first_21_mri_3d'
# iterate over files in that directory
file_list = [os.path.join(data_dir, filename) for filename in os.listdir(data_dir)]
file_names = os.listdir(data_dir)

In [68]:
os.chdir(ROOT_DIR + "dataset-preparation/")

In [69]:
file_list = [os.path.join(data_dir, filename) for filename in os.listdir(data_dir)]


In [70]:
image = nib.load(file_list[0]).get_fdata()
len(pywt.wavedecn(image, "db1")[1].keys())
pywt.wavedecn(image, "db1")[1]["aad"].flatten()

array([ 507967.53355261,       0.        ,  184174.42641593,
             0.        , 1190734.53578099,       0.        ,
       2761963.20013496,       0.        ])

In [71]:
def convert_to_wavelet_basis_3d(folder_dir, basis="db1", normalized = False):
    file_list = [os.path.join(folder_dir, filename) for filename in os.listdir(folder_dir)]
    #Setup df Dict
    image = nib.load(file_list[0]).get_fdata()
    first_image = pywt.wavedecn(image, basis)
    layer_len = len(first_image)
    direction_names = first_image[1].keys()
    direction_num = len(direction_names)
    print(str(layer_len) + " layers being used")
    

    #Fill DF DICT
    layer_arr = [0] * (len(file_list) * (layer_len - 1) * direction_num + len(file_list))
    orientation = [0] * (len(file_list) * (layer_len - 1) * direction_num + len(file_list))
    data_arr = [0] * (len(file_list) * (layer_len - 1) * direction_num + len(file_list))
    cnt = 0
    for k in range(len(file_list)):

        image = np.array(nib.load(file_list[k]).get_fdata())

        if normalized:
            std= np.std(image)
            mean = np.mean(image)
            image = (image- mean)/std

    
        transformed = pywt.wavedecn(image, 'db1')
        

        arr = np.array(transformed[0][0]).flatten()
        layer_arr[cnt] = 1
        orientation[cnt] =  "L1"
        data_arr[cnt] = arr.flatten()
        cnt += 1

        for i in range(1, layer_len): 
            for j in direction_names:
                
                arr = np.array(transformed[i][j]).flatten()
                layer_arr[cnt] = i+1
                orientation[cnt] =  j
                data_arr[cnt] = arr.flatten()
                cnt += 1

    df = pd.DataFrame()

    df["layer"] = layer_arr
    df["orientation"] = orientation
    df["data"] = data_arr
    new_df = pd.DataFrame(columns=["layer", "orientation", "data"])
    for lo, sf in df.groupby(["layer", "orientation"])[["data"]]:#.agg(lambda sf: np.concatenate(sf["Data"].tonumpy()))
        new_df.loc[len(new_df.index)] = [lo[0], lo[1],  np.hstack(sf['data'])]
    
    return new_df

In [72]:
convert_to_wavelet_basis_3d(data_dir)

8 layers being used


,layer,orientation,data
0,1,L1,"[17811885.578976702, 3314538.5449532396, 10796..."
1,2,aad,"[507967.5335526131, 0.0, 184174.42641592678, 0..."
2,2,ada,"[-4897389.359303329, -2489046.852654637, 83082..."
3,2,add,"[1351491.9734704683, 0.0, 636789.4305243948, 0..."
4,2,daa,"[-3792843.6008991404, -1590854.6782308489, -17..."
5,2,dad,"[-225745.61939983652, 0.0, 1111281.4317971624,..."
6,2,dda,"[511702.53095676063, 1099432.0933640988, -5848..."
7,2,ddd,"[231736.35096509475, 0.0, 817569.9163335817, 0..."
8,3,aad,"[276337.17578125047, 871379.3144531258, 0.0, -..."
9,3,ada,"[-1954787.8085937523, -1960706.7324218773, -11..."
